In [ ]:
import argparse, os
import matplotlib.pyplot as plt
import numpy as np
import random as rn
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout
from keras import backend as K
from keras import optimizers
from keras.layers import Input, Dense, Flatten, Lambda, Concatenate, Reshape, \
    TimeDistributed, LSTM, RepeatVector, SimpleRNN, Activation
from keras.models import Model, load_model
from keras.callbacks import TensorBoard
from keras.losses import mse
from keras.utils import plot_model
from numpy import shape

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
        #   'figure.figsize': (19.2, 10.8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

class data_preproc:
    def __init__(self):
        # One motion pattern is as (self.frames_per_pattern, self.points_per_frame, self.features_per_point)
        self.frames_per_pattern     = 10 # For 1 second for 10 fps radar rate
        self.points_per_frame       = 64 # We want to oversample every radar frame to 64 points while keeping the mean and variance the same
        self.features_per_point     = 4  # The radar can provides us (x, y, z, Doppler, RCS), but we only keep the first four feature, i.e. (x, y, z, Doppler)

        # Train and test data split ratio
        self.split_ratio            = 0.8

        # Rotation matrix due to tilt angle
        tilt_angle  = 10.0 # degrees
        self.height = 1.8   # meters
        self.rotation_matrix = np.array([[1.0, 0.0, 0.0],\
                                        [0.0, np.cos(np.deg2rad(tilt_angle)), np.sin(np.deg2rad(tilt_angle))],\
                                        [0.0, -np.sin(np.deg2rad(tilt_angle)), np.cos(np.deg2rad(tilt_angle))]])

    def load_bin(self, binfile_path, fortrain=True):
        # Record centroid history for analysis purpose
        centroidX_his = []
        centroidY_his = []
        centroidZ_his = []

        # Load the recorded bin file
        raw_pointcloud = np.load(binfile_path, allow_pickle=True)

        # Accumulated the motion patterns with (self.frames_per_pattern) frames
        total_pattern = []
        for idx in range(len(raw_pointcloud)-self.frames_per_pattern):
            total_pattern.append(raw_pointcloud[idx : (idx + self.frames_per_pattern)])

        # Original point vector in the .bin file:
        #   [frame number, point ID, target ID, \\ Basic information
        #   centroidX, centroidY, centroidZ, centroidVelocityX, centroidVelocityY, centroidVelocityZ, \\ Centorid information
        #   range, azimuth angle, elevation angle, Doppler, SNR, noise level] \\ Point information
        # Extract the feature vector (delta_x, delta_y, z, D, pointRCS) from the original point vector and do data oversampling proposed in the paper
        total_processed_pattern = []
        total_processed_no_norm = []
        for pattern in total_pattern:
            # Get the centroid information from the very first frame in a pattern and do coordiante transformation
            # As the centroid is already in the radar Cartesian coordinates, we just need to transfer it to the ground Cartesian coordinates
            centroidX, centroidY, centroidZ, centroidVx, centroidVy, centroidVz = pattern[0][0][3], pattern[0][0][4], pattern[0][0][5], pattern[0][0][6], pattern[0][0][7], pattern[0][0][8]
            results      = np.matmul(self.rotation_matrix, np.array([centroidX,centroidY,centroidZ]))
            centroidX    = results[0]
            centroidY    = results[1]
            centroidZ    = results[2] + self.height

            # Record the centroid history over time
            centroidX_his.append(centroidX)
            centroidY_his.append(centroidY)
            centroidZ_his.append(centroidZ)

            processed_pattern  = []
            for frame in pattern:
                processed_frame = []
                for point in frame:
                    # Get the original point information.
                    pointR, pointAZ, pointEL, pointD, pointSNR, pointNoise = point[9], point[10], point[11], point[12], point[13], point[14]

                    # Get the point's position in the Cartesian coord and then do coordiante transformation
                    pointX      = pointR*np.cos(pointEL)*np.sin(pointAZ)
                    pointY      = pointR*np.cos(pointEL)*np.cos(pointAZ)
                    pointZ      = pointR*np.sin(pointEL)
                    results     = np.matmul(self.rotation_matrix, np.array([pointX, pointY, pointZ]))
                    pointX      = results[0]
                    pointY      = results[1]
                    pointZ      = results[2] + self.height

                    # Subtract the point's position from the centroid in the very first frame in a motion pattern
                    delta_x     = pointX - centroidX
                    delta_y     = pointY - centroidY
                    delta_z     = pointZ
                    delta_D     = pointD
                    pointRCS    = 4*10*np.log10(pointR) + pointSNR*0.1 + pointNoise*0.1 # in dBsm

                    # Form the feature vector for each frame
                    feature_vector = [delta_x, delta_y, delta_z, delta_D, pointRCS]
                    processed_frame.append(feature_vector[0:self.features_per_point]) # Only keep 3D spatial info and the Doppler
                processed_pattern.append(processed_frame)
                # Do the data oversampling proposed in the paper
                processed_pattern_oversampled = self.proposed_oversampling(processed_pattern)
            total_processed_pattern.append(processed_pattern_oversampled)

        total_processed_pattern_np = np.array(total_processed_pattern)

        # Train and test split
        split_idx   = int(total_processed_pattern_np.shape[0]*self.split_ratio)
        traindata   = total_processed_pattern_np[0:split_idx]
        testdata    = total_processed_pattern_np[split_idx:]

        if fortrain is True: # For training, need data split to obtain both training and testing dataset
            print("INFO: Total normal motion pattern data shape: " + str(total_processed_pattern_np.shape))
            print("INFO: Training motion pattern data shape" + str(traindata.shape))
            print("INFO: Testing motion pattern data shape" + str(testdata.shape))
            return traindata, testdata
        else: # For inference on anomaly dataset
            print("INFO: Total inference motion pattern data shape: " + str(total_processed_pattern_np.shape))
            return total_processed_pattern, centroidZ_his

    def proposed_oversampling(self, processed_pointcloud):
        # # Check the input
        # point_list = []
        # for frame in processed_pointcloud:
        #     point_list.extend(frame)
        # point_list_np  = np.array(point_list)
        # assert (point_list_np.shape[-1] == self.features_per_point), ("ERROR: Input processed_pointcloud has different feature length rather than %s!" %(self.features_per_point))

        # Do the data oversampling
        processed_pointcloud_oversampled = []
        for frame in processed_pointcloud:
            frame_np = np.array(frame)
            # Check if it's empty frame
            N = self.points_per_frame
            M = frame_np.shape[0]
            assert (M != 0), "ERROR: empty frame detected!"
            # Rescale and padding
            mean        = np.mean(frame_np, axis=0)
            sigma       = np.std(frame_np, axis=0)
            frame_np    = np.sqrt(N/M)*frame_np + mean - np.sqrt(N/M)*mean # Rescale
            frame_oversampled = frame_np.tolist()
            frame_oversampled.extend([mean]*(N-M)) # Padding with mean
            # # Check if mean and sigma keeps the same. Comment for saving time.
            # new_mean    = np.mean(np.array(frame_oversampled), axis=0)
            # new_sigma   = np.std(np.array(frame_oversampled), axis=0)
            # assert np.sum(np.abs(new_mean-mean))<1e-5, ("ERROR: Mean rescale and padding error!")
            # assert np.sum(np.abs(new_sigma-sigma))<1e-5, ("ERROR: Sigma rescale and padding error!")
            processed_pointcloud_oversampled.append(frame_oversampled)

        processed_pointcloud_oversampled_np = np.array(processed_pointcloud_oversampled)
        assert (processed_pointcloud_oversampled_np.shape[-2] == self.points_per_frame), ("ERROR: The new_frame_data has different number of points per frame rather than %s!" %(self.points_per_frame))
        assert (processed_pointcloud_oversampled_np.shape[-1] == self.features_per_point), ("ERROR: The new_frame_data has different feature length rather than %s!" %(self.features_per_point))

        return processed_pointcloud_oversampled_np



/home/mart/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-03-18 16:11:43.807011: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 16:11:43.848214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 16:11:43.848260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 16:11:43.850196: E external/local_xla/xla/stream_ex

In [2]:
if __name__ == '__main__':
    project_path = '/home/mart/mmfall/'
print("INFO: Load train/test data...")
train_data      = np.load(project_path + 'data/normal_train_data.npy', allow_pickle=True)
test_data       = np.load(project_path + 'data/normal_test_data.npy', allow_pickle=True)
shape(train_data)

INFO: Load train/test data...


(60816, 10, 64, 4)

In [3]:

# Positional Encoding Layer
class PositionalEncoding(layers.Layer):
    def __init__(self, sequence_size, embedding_dim, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.sequence_size = sequence_size
        self.embedding_dim = embedding_dim

    def build(self, input_shape):
        self.positional_embeddings = self.add_weight(
            name='pos_embeddings',
            shape=(self.sequence_size, self.embedding_dim),
            initializer='uniform',
            trainable=True,
        )

    def call(self, x):
        return x + self.positional_embeddings

# Transformer Encoder Layer
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout_rate):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate)(x, x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Add()([inputs, x])

    # Feed Forward Network
    ff_network = tf.keras.Sequential([
        layers.Dense(ff_dim, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(inputs.shape[-1]),
    ])

    x = layers.LayerNormalization(epsilon=1e-6)(x)
    ff_output = ff_network(x)
    x = layers.Add()([x, ff_output])
    return x

# Transformer Model for Anomaly Detection
def build_transformer_model(sequence_size, embedding_dim, head_size, num_heads, ff_dim, dropout_rate):
    inputs = layers.Input(shape=(sequence_size, embedding_dim))
    x = PositionalEncoding(sequence_size, embedding_dim)(inputs)
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout_rate)

    # Reconstruction of the input
    outputs = layers.Dense(embedding_dim)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model



In [4]:
# Model Configuration
sequence_size = 10 * 64  # Flatten the spatial dimensions (10 frames, 64 points)
embedding_dim = 4       # Features dimension
head_size = 16          # Size of each attention head
num_heads = 4           # Number of attention heads
ff_dim = 8            # Hidden layer size in feed forward network
dropout_rate = 0.1      # Dropout rate

# Build and compile the model
model = build_transformer_model(sequence_size, embedding_dim, head_size, num_heads, ff_dim, dropout_rate)
model.compile(optimizer='adam', loss='mse')

# Model summary
model.summary()

# Reshape the training data to fit the model input
train_data_reshaped = train_data.reshape(-1, sequence_size, embedding_dim)

# Train the model
model.fit(train_data_reshaped, train_data_reshaped, epochs=1, batch_size=8)

# After training, you can use the reconstruction error to detect anomalies in new data.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 640, 4)]             0         []                            
                                                                                                  
 positional_encoding (Posit  (None, 640, 4)               2560      ['input_1[0][0]']             
 ionalEncoding)                                                                                   
                                                                                                  
 layer_normalization (Layer  (None, 640, 4)               8         ['positional_encoding[0][0]'] 
 Normalization)                                                                                   
                                                                                              

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


KeyboardInterrupt: 

In [4]:
#model.save('/home/mart/mmfall/saved_model/mymodel1', save_format='tf')
loaded_model = tf.keras.models.load_model('/home/mart/mmfall/saved_model/mymodel1')
sequence_size = 10 * 64  # Flatten the spatial dimensions (10 frames, 64 points)
embedding_dim = 4       # Features dimension
head_size = 16          # Size of each attention head
num_heads = 4           # Number of attention heads
ff_dim = 8            # Hidden layer size in feed forward network
dropout_rate = 0.1      # Dropout rate


2024-03-18 16:12:06.815147: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-03-18 16:12:06.949158: W tensorflow/c/c_api.cc:305] Operation '{name:'AssignVariableOp_28' id:423 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_28}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](multi_head_attention/attention_output/bias/v, Identity_28)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [5]:
def calculate_reconstruction_error(model, data):
    # Predict the reconstruction of the input data
    reconstructed_data = model.predict(data)
    # Calculate the mean squared error between the original and reconstructed data
    mse = tf.keras.losses.mean_squared_error(data, reconstructed_data)
    # Reduce to get the mean error for the batch
    error = tf.reduce_mean(mse)
    return K.eval(error)  # Use K.eval() to convert the tensor to a numpy array

test_data_reshaped = test_data.reshape(-1, sequence_size, embedding_dim)
output = calculate_reconstruction_error(loaded_model, test_data_reshaped)



/home/mart/.local/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-03-18 16:12:34.727545: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_2/BiasAdd' id:706 op device:{requested: '', assigned: ''} def:{{{node dense_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2/Tensordot, dense_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-03-18 16:12:34.991177: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 209715200 exceeds 10% of free system memory.
2024-03-18 16:12:35.193841: W external/local_tsl/tsl/framew

: 

In [54]:
class autoencoder_mdl:
    def __init__(self, model_dir):
        self.model_dir = model_dir


    def transmodel_train(self, train_data, test_data):
    #Input
        n_frames = 10
        n_points = 64
        n_features = 4
        n_latentdim = 512 #todo why am I using these numbers?
        n_heads = 4  
        inputs = Input(shape=(n_frames, n_points, n_features))
        outputs = Input(shape=(n_frames, n_points, n_features))

        #positional embedding
        def positional_encoding(max_seq_length, d_model):
            pos_enc = np.zeros((max_seq_length, d_model))
            position = np.arange(0, max_seq_length)[:, np.newaxis]
            div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
            pos_enc[:, 0::2] = np.sin(position * div_term)
            pos_enc[:, 1::2] = np.cos(position * div_term)
            return tf.convert_to_tensor(pos_enc, dtype=tf.float32)

        max_seq_length = 10
        d_model = 512
        self.pos_enc = positional_encoding(max_seq_length, d_model)
        input_flatten = TimeDistributed(Flatten())(inputs)
        def transformer_encoder(inputs):
            print("inside encoder")
            print("Shape of input flatten:", input_flatten.shape)
            
            attention_output = MultiHeadAttention(num_heads=n_heads, key_dim=n_latentdim)(input_flatten, input_flatten) #multihead
            attention_output = LayerNormalization(epsilon=1e-6)(attention_output + input_flatten) #adding and normalization with residual connection
            ffn_output = TimeDistributed(Dense(256, activation='relu'))(attention_output) #feed forward network
            ffn_output = LayerNormalization(epsilon=1e-6)(ffn_output + attention_output) #add & norm again
            print("ffn_outputshape", ffn_output.shape)
            print("inputsshape", inputs.shape)
            return ffn_output
       
        encoder_output = transformer_encoder(inputs)
        Z_mean = TimeDistributed(Dense(n_latentdim, activation=None), name='qzx_mean')(encoder_output)
        Z_log_var = TimeDistributed(Dense(n_latentdim, activation=None), name='qzx_log_var')(encoder_output)


        #building the model
        self.TRANS_mdl = Model(inputs, outputs)
        print(self.TRANS_mdl.summary())
        self.TRANS_mdl.compile(optimizer=adam, loss=mse)
        # Train the model


        self.TRANS_mdl.fit(train_data, train_data, # Train on the normal training set
                epochs=5,
                batch_size=80,
                shuffle=False,
                validation_data=(test_data, test_data), # Testing on the normal tesing dataset
                callbacks=[TensorBoard(log_dir=(self.model_dir + "/../model_history/TRANS_online"))])
        self.TRANS_mdl.save(self.model_dir + 'RAE_TRANS_online.h5')
       
        print("INFO: Training is done!")
        print("*********************************************************************")

model = autoencoder_mdl(model_dir = (project_path + 'saved_model/'))

model.transmodel_train(train_data, test_data)

inside encoder
Shape of input flatten: (None, 10, 256)
ffn_outputshape (None, 10, 256)
inputsshape (None, 10, 64, 4)
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 10, 64, 4)]          0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, 10, 64, 4)]          0         []                            
                                                                                                  
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
__________________________________________________________________________________________________
None


NameError: name 'adam' is not defined

In [42]:
from tensorflow.keras.optimizers import Adam
class TransformerAnomalyDetection(tf.keras.Model):
    def __init__(self, num_heads, d_model, num_layers, d_ff, input_shape_):
        super(TransformerAnomalyDetection, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.num_layers = num_layers
        self.d_ff = d_ff
        self.input_shape_ = input_shape_

        # Positional encoding
        # Transformer layers
        self.transformer_layers = [
            self._get_transformer_layer() for _ in range(num_layers)
        ]

        # Output layer
        self.output_layer = Dense(1, activation='sigmoid')

    def _get_positional_encoding(self,max_seq_length, d_model):
        pos_enc = np.zeros((max_seq_length, d_model))
        position = np.arange(0, max_seq_length)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        pos_enc[:, 0::2] = np.sin(position * div_term)
        pos_enc[:, 1::2] = np.cos(position * div_term)
        return tf.convert_to_tensor(pos_enc, dtype=tf.float32)

    def _get_transformer_layer(self):
        return tf.keras.Sequential([
            MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model, value_dim=self.d_model),  # Add value_dim argument
            LayerNormalization(epsilon=1e-6),
            Dense(self.d_ff, activation='relu'),
            Dense(self.d_model),
            LayerNormalization(epsilon=1e-6)
        ])


    def call(self, inputs):
        # Add positional encoding
        max_seq_length = 64
        d_model = 4
        inputs += self._get_positional_encoding(max_seq_length, d_model)

        # Pass through transformer layers
        for layer in self.transformer_layers:
            inputs = layer(inputs)

        # Aggregate across time steps
        aggregated = tf.reduce_mean(inputs, axis=1)

        # Output layer
        output = self.output_layer(aggregated)
        return output

input_shape = (shape(train_data))  # 4-dimensional point cloud data
model = TransformerAnomalyDetection(num_heads=4, d_model=64, num_layers=2, d_ff=128, input_shape_=input_shape)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy')
X_train = train_data  # Fix: change x_train to X_train
num_epochs = 10
batch_size = 16
model.fit(X_train, epochs=num_epochs, batch_size=batch_size)
model.summary()

TypeError: in user code:

    File "/tmp/ipykernel_482935/190172261.py", line 46, in call  *
        inputs = layer(inputs)
    File "/home/mart/.local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    TypeError: MultiHeadAttention.call() missing 1 required positional argument: 'value'


In [ ]:
# After training, you can use the encoder to reconstruct the test data
# x_test = ...
# reconstructed = ??.predict(x_test)
reconstructed = TRANS_mdl.predict(inferencedata)
# Calculate the mean squared error of the reconstruction
# mse = np.mean(np.power(x_test - reconstructed, 2), axis=1)
# A high MSE indicates an anomaly. You can set a threshold MSE value to flag anomalies.

In [ ]:

print("INFO: Start HVRAE/HVRAE_SL/RAE model training and testing...")
model = autoencoder_mdl(model_dir = (project_path + 'saved_model/'))
model.transmodel_train(train_data, test_data)


# Load inference dataset and the ground truth timesheet
inferencedata_file                      = project_path + 'data/DS1/DS1_4falls'
inferencedata, centroidZ_history        = data_preproc().load_bin(inferencedata_file + '.npy', fortrain=False)
# Ground truth time index file exists
if os.path.exists(inferencedata_file + '.csv'): 
    gt_falls_idx                        = np.genfromtxt(inferencedata_file + '.csv', delimiter=',').astype(int)
# Maybe this file doesn't contain any falls
else: 
    gt_falls_idx                        = []

# Load the models
model                                   = autoencoder_mdl(model_dir = project_path + 'saved_model/')

HVRAE_loss_history                       = model.TRANS_predict(inferencedata)
plt.figure()
plt.ylim(0.0, 2.2)
plt.xlabel("Time in Seconds")
time_step = np.arange(0, len(centroidZ_history))*0.1 # 0.1 seconds per frame
plt.plot(time_step, centroidZ_history, linewidth=2, label='Centroid Height')
plt.plot(time_step, HVRAE_loss_history, linewidth=2, label='Anomaly Level')
plt.legend(loc="upper right")
plt.title('HVRAE Results')
plt.savefig(inferencedata_file+'_HVRAE_prediction.png')
plt.show()